# Analysis Template: Inhibition
Updated 5/6/24 DFM

In [1]:
#enables autoreloding of modules
%load_ext autoreload
%autoreload 2

from htbam_db_api.htbam_db_api import LocalHtbamDBAPI
from htbam_analysis.analysis.experiment import HTBAMExperiment

#enable inline plotting of matplotlib figures
%matplotlib inline

#set the figure format to SVG
%config InlineBackend.figure_format = 'svg'

## 1. Connect DB Api

In [2]:
### PARAMETERS:
EGFP_SLOPE = 91900.03
EGFP_SLOPE_CONC_UNITS = 'nM' #RFU/nM

root = './ic50_data/'
db_conn = LocalHtbamDBAPI(
    standard_curve_data_path= root + 'd3_2_StandardSeries_Analysis.csv',
    standard_name="standard_5-FAM", 
    standard_substrate="5-FAM", 
    standard_units="uM",
    kinetic_data_path= root+ 'd3_TitrationSeries_Analysis.csv',
    kinetic_name="kinetics_AVI4516", 
    kinetic_substrate="NSP 4/5", 
    kinetic_units="nM")

htbam_experiment = HTBAMExperiment(db_conn)


Connected to database.
Experiment found with the following runs:
['standard_5-FAM', 'kinetics_AVI4516']


## 2. Standards

In [3]:
from htbam_analysis.analysis.fitting import fit_luminance_vs_concentration

# For demonstration purposes, I'm doing this in 3 lines
# we can either keep it "exposed" like this (might be useful?)
# Or provide a simple wrapper fx that looks like:
# htbam_experiment.fit_standard_curve(experiment_name = 'standard_5-FAM', analysis_name='htbam_experiment')

standard_experiment_data = htbam_experiment.get_run('standard_5-FAM')       # retrieve our raw data
standard_fits = fit_luminance_vs_concentration(standard_experiment_data)    # perform a fit
htbam_experiment.set_run('standard_5-FAM_fits', standard_fits)              # save the fit results

Existing run data not found. Fetching from database.
Fit slopes for 1792 wells.
Elapsed 1.107 seconds.


In [4]:
htbam_experiment.plot_standard_curve_chip('standard_5-FAM_fits', 'standard_5-FAM')

## 3. Fit Initial Rates

In [5]:
from htbam_analysis.analysis.fitting import fit_luminance_vs_time

kinetics_experiment_data = htbam_experiment.get_run('kinetics_AVI4516')       # retrieve our raw data
kinetics_fits = fit_luminance_vs_time(kinetics_experiment_data, start_timepoint = 1, end_timepoint=10)      # perform a fit, skipping the first timepoint
htbam_experiment.set_run('kinetics_AVI4516_fits', kinetics_fits)                # save the fit results

Existing run data not found. Fetching from database.
Fit slopes for 1792 wells at 12 concentrations.
Elapsed 0.076 seconds.


In [6]:
htbam_experiment.plot_initial_rates_chip('kinetics_AVI4516_fits', 'kinetics_AVI4516')#, remove_0_point=True) # plot the initial rates

# N.F. This is how far I got as of 5/16/25

## 4. Filter initial rates

In [ ]:
htbam_experiment.compute_enzyme_concentration('kinetic_0', EGFP_SLOPE)

In [ ]:
htbam_experiment.filter_initial_rates('kinetic_0',
                                      'standard_0',
                                      standard_curve_r2_cutoff = 0.98,
                                      expression_threshold = 1.0,
                                      initial_rate_R2_threshold = 0.0, 
                                      positive_initial_slope_filter = False)

In [ ]:
htbam_experiment.plot_filtered_initial_rates_chip('kinetic_0')

## 5. Fit Inhibition Constant:

In [ ]:
# NOTE: This will matter for michaelis-menten, but I am leaving it for now to fit IC50s
# substrate_conc_unit = db_conn._json_dict['runs']['kinetics_0']['conc_unit']
# if  substrate_conc_unit != EGFP_SLOPE_CONC_UNITS: print('Substrate concentration units do not match EGFP standard curve units! \n{} != {}'.format(substrate_conc_unit, EGFP_SLOPE_CONC_UNITS))

# unit_converstion = 0.001 #convert FROM eGFP units TO substrate units (in this case, nM to uM)
# enzyme_concentration_converted_units = enzyme_concentration * unit_converstion

# #Double check!
# print('Conversion:')
# print('{} {} = {} {}  ?'.format(enzyme_concentration[0], EGFP_SLOPE_CONC_UNITS, enzyme_concentration_converted_units[0], substrate_conc_unit))

In [ ]:
htbam_experiment.fit_ic50s('kinetic_0')

In [ ]:
htbam_experiment.filter_ic50s('kinetic_0',
                              z_score_threshold_ic50 = 1.5,
                              z_score_threshold_expression = 1.5,
                               save_intermediate_data = False)

In [ ]:
htbam_experiment.plot_filtered_ic50('kinetic_0', show_average_fit=True)

## 6. Export to CSV

In [ ]:
htbam_experiment.export_ic50_result_csv(path_to_save=root,
                                        run_name='kinetics_0')